## Problem 1 

Find, by the use of Figure 1, the direct kinematic transformations, T0
4 for the
robot stylus, and T0
5 for the robot camera, as function of all joint angles.

In [33]:
# Import libraries

import sympy as sp
from sympy import init_printing
from sympy import pprint
import numpy as np

theta1 = sp.symbols('theta1')
theta2 = sp.symbols('theta2')
theta3 = sp.symbols('theta3')
theta4 = sp.symbols('theta4')


# DH Transformation Matrix

def dh_matrix(theta, d, a, alpha):
    """
    Standard DH:
    A = Rot_z(theta) * Trans_z(d) * Trans_x(a) * Rot_x(alpha)
    """
    ct = sp.cos(theta)
    st = sp.sin(theta)
    ca = sp.cos(alpha)
    sa = sp.sin(alpha)

    return sp.Matrix([
        [ct, -st * ca,  st * sa, a * ct],
        [st,  ct * ca, -ct * sa, a * st],
        [0,      sa,      ca,     d   ],
        [0,      0,       0,      1   ]
    ])

In [34]:

# Calculate A1 to A 4

a1 = dh_matrix(theta1, 50, 0, sp.pi/2)
a2 = dh_matrix(theta2 , 0, 93, 0)
a3 = dh_matrix(theta3, 0, 93, 0)
a4 = dh_matrix(theta4, 0, 50, 0)
print (f'A1: {a1}')
print (f'A2: {a2}')
print (f'A3: {a3}')
print (f'A4: {a4}')

a5 = sp.Matrix([[1, 0, 0, -15], [0, 1, 0, 45], [0, 0, 1, 0], [0, 0, 0, 1]])
init_printing(a4)

A1: Matrix([[cos(theta1), 0, sin(theta1), 0], [sin(theta1), 0, -cos(theta1), 0], [0, 1, 0, 50], [0, 0, 0, 1]])
A2: Matrix([[cos(theta2), -sin(theta2), 0, 93*cos(theta2)], [sin(theta2), cos(theta2), 0, 93*sin(theta2)], [0, 0, 1, 0], [0, 0, 0, 1]])
A3: Matrix([[cos(theta3), -sin(theta3), 0, 93*cos(theta3)], [sin(theta3), cos(theta3), 0, 93*sin(theta3)], [0, 0, 1, 0], [0, 0, 0, 1]])
A4: Matrix([[cos(theta4), -sin(theta4), 0, 50*cos(theta4)], [sin(theta4), cos(theta4), 0, 50*sin(theta4)], [0, 0, 1, 0], [0, 0, 0, 1]])


In [35]:
# Multiply to get T0_4
t0_4 = a1 * a2 * a3 * a4 
t0_5 = t0_4 * a5
print(f'T0_4: {t0_4}')
print(f'T0_5: {t0_5}')

T04_simpl = sp.simplify(sp.trigsimp(t0_4, method='fu'))
T05_simpl = sp.simplify(sp.trigsimp(t0_5, method='fu'))
sp.pprint(T04_simpl[:,3])

T0_4: Matrix([[(-sin(theta2)*sin(theta3)*cos(theta1) + cos(theta1)*cos(theta2)*cos(theta3))*cos(theta4) + (-sin(theta2)*cos(theta1)*cos(theta3) - sin(theta3)*cos(theta1)*cos(theta2))*sin(theta4), -(-sin(theta2)*sin(theta3)*cos(theta1) + cos(theta1)*cos(theta2)*cos(theta3))*sin(theta4) + (-sin(theta2)*cos(theta1)*cos(theta3) - sin(theta3)*cos(theta1)*cos(theta2))*cos(theta4), sin(theta1), 50*(-sin(theta2)*sin(theta3)*cos(theta1) + cos(theta1)*cos(theta2)*cos(theta3))*cos(theta4) + 50*(-sin(theta2)*cos(theta1)*cos(theta3) - sin(theta3)*cos(theta1)*cos(theta2))*sin(theta4) - 93*sin(theta2)*sin(theta3)*cos(theta1) + 93*cos(theta1)*cos(theta2)*cos(theta3) + 93*cos(theta1)*cos(theta2)], [(-sin(theta1)*sin(theta2)*sin(theta3) + sin(theta1)*cos(theta2)*cos(theta3))*cos(theta4) + (-sin(theta1)*sin(theta2)*cos(theta3) - sin(theta1)*sin(theta3)*cos(theta2))*sin(theta4), -(-sin(theta1)*sin(theta2)*sin(theta3) + sin(theta1)*cos(theta2)*cos(theta3))*sin(theta4) + (-sin(theta1)*sin(theta2)*cos(theta3

## Problem 2
Determine the inverse kinematic transformation

In [36]:
# x4 = T04_simpl[:3,0]
# o4 = T04_simpl[:3,3]

x4 = t0_4[:3,0]
o4 = t0_4[:3,3]

o4x = o4[0,0]
o4y = o4[1,0]
o4z = o4[2,0]

In [45]:
# Find q1-q4

def inverse_kinematics(o4x, o4y, o4z): 

    q1 = sp.atan2(o4y, o4x)

    r = sp.sqrt(o4x**2 + o4y**2)
    s = o4z - 50
    c3 = (r**2 + s**2 - 93**2 - 93**2) / (2 * 93 * 93)
    q3 = sp.atan2(sp.sqrt(1 - c3**2), c3)

    s3 = sp.sin(q3)
    q2 = sp.atan2(s, r) - sp.atan2(93 * s3, 93 + 93 * c3)

    q4 = -(q2 + q3)

    return (np.array([q1, q2, q3, q4]))



## Problem 3

In [ ]:
R = 32 # radius
pc = np.array([150, 0, 120]) # mm 

configs = []

for j in range(37):
    phi = 2*np.pi*j/36

    o4x = pc[0] + 0
    o4y = pc[1] + R*np.cos(phi)
    o4z = pc[2] + R*np.sin(phi)

    configs.append(inverse_kinematics(o4x, o4y, o4z))

for i in configs:
    print (i)




[0.210182586662170 -0.00790522340841759 0.872131723787321
 -0.864226500378903]
[0.207080613842599 0.0520746056832362 0.811834186447723 -0.863908792130959]
[0.197845287731223 0.111722299224034 0.749873098816843 -0.861595398040877]
[0.182692045817031 0.169506588731593 0.687727608059656 -0.857234196791249]
[0.161990846336244 0.223789356086055 0.627205062325115 -0.850994418411170]
[0.136278069768380 0.272773039973769 0.570570096461700 -0.843343136435469]
[0.106264862891079 0.314453950807469 0.520667713016386 -0.835121663823855]
[0.0728352271115261 0.346637610822505 0.480927487255077 -0.827565098077582]
[0.0370280125712929 0.367126977404697 0.455023747941660 -0.822150725346357]
[1.30628991909051e-17 0.374182995620479 0.445987324944397
 -0.820170320564876]
[-0.0370280125712929 0.367126977404697 0.455023747941660
 -0.822150725346357]
[-0.0728352271115261 0.346637610822506 0.480927487255076
 -0.827565098077582]
[-0.106264862891079 0.314453950807469 0.520667713016386 -0.835121663823855]
[-0.136